# Population Density by Municipality

This notebook computes **population density** (inhabitants per km²) for each Spanish
municipality and year using cleaned population data from the municipal census
(*Padrón Municipal*) and official municipal boundaries from the National Geographic
Institute (CNIG).

Population density is generated as a **derived demographic attribute** at the
municipality–year level and exported as a standalone dataset to ensure
methodological clarity, traceability, and reproducibility.

The resulting dataset will later be integrated with other demographic, services,
agricultural, and land-use indicators.

## Definition

Population density measures the number of inhabitants per square kilometer in a
given territorial unit.

**Formula:**

$$
\text{Population Density} = \frac{\text{Total Population}}{\text{Area (km}^2\text{)}}
$$

This indicator is fundamental for understanding settlement patterns, territorial
pressure, and rural depopulation dynamics.

**Sources:**  
- Population data: INE – *Padrón Municipal de Habitantes*  
- Municipal boundaries: CNIG – *Centro de Descargas*  
  https://centrodedescargas.cnig.es/CentroDescargas/home

In [ ]:
"""
Notebook: 02_demography_population_density.ipynb
Purpose: Compute population density per municipality and year
Input: 01_padron_clean_1996_2024.csv, CNIG municipal shapefiles
Output: demography_population_density_1996_2024.csv
Author: Juan Zotes
Last updated: 2026-02-03
"""

## 1. Load required libraries and configure environment

This step loads all necessary libraries for:
- Downloading and extracting spatial data from CNIG
- Processing shapefiles and computing geometric areas
- Merging spatial and demographic datasets

In [ ]:
# Standard library
from pathlib import Path
import tempfile
import zipfile
import requests
import warnings

# Third-party libraries
import pandas as pd
import geopandas as gpd

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [ ]:
# Base data directory (portable across Windows, Linux, Codespaces)
DATA_DIR = Path(
    r"/workspaces/rural-migration-land-use-spain/data/demography/processed"
)

# Directory for derived outputs
DERIVED_DIR = Path(
    r"/workspaces/rural-migration-land-use-spain/data/demography/derived"
)

# Ensure directories exist
DATA_DIR.mkdir(parents=True, exist_ok=True)
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {DERIVED_DIR}")

## 2. Download official municipal boundaries from CNIG

Municipal shapefiles are downloaded directly from the National Geographic Institute
(CNIG) to ensure reproducibility and avoid local storage requirements.

Two separate files are required:
- **Peninsula + Balearic Islands** (ETRS89 projection)
- **Canary Islands** (REGCAN95 projection)

Both datasets are downloaded, extracted, and temporarily stored for processing.

In [ ]:
# CNIG download URLs for municipal boundaries (2024)
# Source: https://centrodedescargas.cnig.es/CentroDescargas/home

PENINSULA_URL = (
    "https://centrodedescargas.cnig.es/CentroDescargas/descargaDir?"
    "codSerie=CAANE&codExp=CAANE_SHP_CCAA_2024"
)

CANARIAS_URL = (
    "https://centrodedescargas.cnig.es/CentroDescargas/descargaDir?"
    "codSerie=CAANE&codExp=CAANE_SHP_CCAA_2024"
)

print("URLs configured for CNIG municipal boundaries download")

In [ ]:
def download_and_extract_shapefile(url, extract_to):
    """
    Download a zipped shapefile from CNIG and extract it to a temporary directory.
    
    Parameters:
    -----------
    url : str
        Direct download URL from CNIG
    extract_to : Path
        Directory where the shapefile will be extracted
    
    Returns:
    --------
    Path
        Path to the extracted shapefile (.shp)
    """
    print(f"Downloading from CNIG...")
    
    # Download the zip file
    response = requests.get(url, stream=True, timeout=300)
    response.raise_for_status()
    
    # Save to temporary file
    zip_path = extract_to / "temp_download.zip"
    with open(zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    print(f"Extracting shapefile...")
    
    # Extract all files
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Find the .shp file
    shp_files = list(extract_to.glob('**/*.shp'))
    
    if not shp_files:
        raise FileNotFoundError("No .shp file found in the downloaded archive")
    
    # Return the first .shp file found (should be the municipalities layer)
    shp_path = shp_files[0]
    print(f"Shapefile extracted: {shp_path.name}")
    
    return shp_path

**Note:** The CNIG download URLs above are placeholders. The actual URLs need to be
verified from the CNIG download center, as they may change over time.

For manual download if automated approach fails:
1. Visit: https://centrodedescargas.cnig.es/CentroDescargas/home
2. Navigate to: Cartografía → España → Límites municipales
3. Download both Peninsula+Baleares and Canarias shapefiles
4. Place them in a `data/spatial/raw/` directory and modify the code accordingly

## 3. Load and process municipal boundaries

The downloaded shapefiles contain municipality polygons with attributes including
the NATCODE field. The last 5 digits of NATCODE correspond to the Mun_Code used in
the demographic dataset.

Steps:
1. Load both shapefiles (Peninsula+Balearic and Canary Islands)
2. Reproject to a common CRS (ETRS89 / UTM zone 30N - EPSG:25830)
3. Extract Mun_Code from NATCODE
4. Compute area in km²
5. Merge both datasets

In [ ]:
# Create temporary directory for downloaded shapefiles
temp_dir = Path(tempfile.mkdtemp())
print(f"Temporary directory: {temp_dir}")

# Download Peninsula + Balearic Islands shapefile
# NOTE: Replace with actual working URL or use manual download
peninsula_dir = temp_dir / "peninsula"
peninsula_dir.mkdir(exist_ok=True)

print("\n=== Downloading Peninsula + Balearic Islands ===")
# peninsula_shp = download_and_extract_shapefile(PENINSULA_URL, peninsula_dir)

# Download Canary Islands shapefile
canarias_dir = temp_dir / "canarias"
canarias_dir.mkdir(exist_ok=True)

print("\n=== Downloading Canary Islands ===")
# canarias_shp = download_and_extract_shapefile(CANARIAS_URL, canarias_dir)

### Alternative: Manual shapefile loading

If automated download fails, manually download the shapefiles and use the following
code instead:

In [ ]:
# ALTERNATIVE: Manual file paths (comment out if using automated download)
# Adjust these paths to match your local file structure

SPATIAL_DIR = Path(r"/workspaces/rural-migration-land-use-spain/data/spatial/raw")

peninsula_shp = SPATIAL_DIR / "recintos_municipales_inspire_peninbal_etrs89.shp"
canarias_shp = SPATIAL_DIR / "recintos_municipales_inspire_canarias_regcan95.shp"

# Verify files exist
if peninsula_shp.exists() and canarias_shp.exists():
    print("✓ Shapefiles found locally")
else:
    print("✗ Shapefiles not found. Please download manually or check paths.")

In [ ]:
# Load Peninsula + Balearic Islands
print("Loading Peninsula + Balearic Islands shapefile...")
gdf_peninsula = gpd.read_file(peninsula_shp)

print(f"Loaded {len(gdf_peninsula)} municipalities (Peninsula + Balearic)")
print(f"CRS: {gdf_peninsula.crs}")
print(f"Columns: {list(gdf_peninsula.columns)}")

In [ ]:
# Load Canary Islands
print("Loading Canary Islands shapefile...")
gdf_canarias = gpd.read_file(canarias_shp)

print(f"Loaded {len(gdf_canarias)} municipalities (Canary Islands)")
print(f"CRS: {gdf_canarias.crs}")
print(f"Columns: {list(gdf_canarias.columns)}")

In [ ]:
# Reproject both to ETRS89 / UTM zone 30N (EPSG:25830)
# This CRS is suitable for area calculations in Spain

TARGET_CRS = "EPSG:25830"

print(f"Reprojecting to {TARGET_CRS}...")

gdf_peninsula = gdf_peninsula.to_crs(TARGET_CRS)
gdf_canarias = gdf_canarias.to_crs(TARGET_CRS)

print("✓ Reprojection complete")

## 4. Extract municipality codes and compute areas

The NATCODE field in the shapefile contains an 11-digit code. The last 5 digits
correspond to the Mun_Code used in the demographic dataset.

Example:
- NATCODE: `34010404001`
- Mun_Code: `04001`

In [ ]:
def process_municipalities(gdf):
    """
    Extract Mun_Code from NATCODE and compute area in km².
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Input geodataframe with NATCODE field
    
    Returns:
    --------
    GeoDataFrame
        Processed geodataframe with Mun_Code and Area_km2 columns
    """
    # Extract last 5 digits of NATCODE
    gdf['Mun_Code'] = gdf['NATCODE'].astype(str).str[-5:]
    
    # Compute area in km² (geometry is in meters after reprojection)
    gdf['Area_km2'] = gdf.geometry.area / 1_000_000
    
    return gdf

In [ ]:
# Process both datasets
gdf_peninsula = process_municipalities(gdf_peninsula)
gdf_canarias = process_municipalities(gdf_canarias)

print("Sample from Peninsula:")
print(gdf_peninsula[['NATCODE', 'NAMEUNIT', 'Mun_Code', 'Area_km2']].head())

In [ ]:
# Merge both datasets
gdf_all = pd.concat([gdf_peninsula, gdf_canarias], ignore_index=True)

print(f"\nTotal municipalities: {len(gdf_all)}")
print(f"Unique Mun_Codes: {gdf_all['Mun_Code'].nunique()}")

In [ ]:
# Create area lookup table (Mun_Code → Area_km2)
area_lookup = gdf_all[['Mun_Code', 'Area_km2']].copy()

# Check for duplicates (should not exist)
duplicates = area_lookup[area_lookup.duplicated(subset=['Mun_Code'], keep=False)]

if len(duplicates) > 0:
    print(f"⚠ Warning: {len(duplicates)} duplicate Mun_Codes found")
    print(duplicates)
else:
    print("✓ No duplicate municipality codes")

## 5. Load cleaned demographic data

This step loads the cleaned historical municipal census dataset, which serves as
the demographic base for density calculations.

In [ ]:
# Load cleaned municipal census data
padron_file = DATA_DIR / "01_padron_clean_1996_2024.csv"

df = pd.read_csv(
    padron_file,
    dtype={"Mun_Code": str}
)

print(f"Loaded {len(df)} records")
print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"Unique municipalities: {df['Mun_Code'].nunique()}")

df.head()

In [ ]:
# Filter for Total population only (exclude sex-disaggregated data)
df_total = df[df['Cat'] == 'Total'].copy()

print(f"Records after filtering for Total category: {len(df_total)}")
df_total.head()

## 6. Merge population data with municipal areas

The area lookup table is merged with the population dataset to enable density
calculations for each municipality-year combination.

In [ ]:
# Merge population data with area data
df_merged = df_total.merge(
    area_lookup,
    on='Mun_Code',
    how='left',
    validate='m:1'  # Many population records to one area record
)

print(f"Merged dataset size: {len(df_merged)}")
df_merged.head()

In [ ]:
# Check for municipalities without area data
missing_area = df_merged[df_merged['Area_km2'].isna()]

if len(missing_area) > 0:
    print(f"⚠ Warning: {missing_area['Mun_Code'].nunique()} municipalities missing area data")
    print("\nMunicipalities without area:")
    print(missing_area[['Mun_Code', 'Mun']].drop_duplicates())
else:
    print("✓ All municipalities have area data")

## 7. Compute population density

Population density is calculated as the total population divided by the municipal
area in km². The result is expressed in inhabitants per km².

In [ ]:
# Compute population density (inhabitants/km²)
df_merged['Pop_Density'] = df_merged['Pop'] / df_merged['Area_km2']

# Handle cases where population is NaN (preserve NaN)
# Handle cases where area is 0 or NaN (set density to NaN)
df_merged.loc[df_merged['Area_km2'] <= 0, 'Pop_Density'] = float('nan')

print("Population density computed")
print(f"\nDensity statistics:")
print(df_merged['Pop_Density'].describe())

## 8. Quality Control

We inspect basic statistics and extreme values to ensure the indicator behaves as
expected across municipalities and years.

In [ ]:
# Check municipalities with highest and lowest density
print("=== Municipalities with highest density (2024) ===")
high_density = df_merged[df_merged['Year'] == 2024].nlargest(10, 'Pop_Density')
print(high_density[['Mun_Code', 'Mun', 'Pop', 'Area_km2', 'Pop_Density']])

print("\n=== Municipalities with lowest density (2024) ===")
low_density = df_merged[df_merged['Year'] == 2024].nsmallest(10, 'Pop_Density')
print(low_density[['Mun_Code', 'Mun', 'Pop', 'Area_km2', 'Pop_Density']])

In [ ]:
# Check for negative or infinite density values (should not exist)
negative_density = df_merged[df_merged['Pop_Density'] < 0]
infinite_density = df_merged[df_merged['Pop_Density'] == float('inf')]

if len(negative_density) > 0:
    print(f"⚠ Warning: {len(negative_density)} records with negative density")
else:
    print("✓ No negative density values")

if len(infinite_density) > 0:
    print(f"⚠ Warning: {len(infinite_density)} records with infinite density")
else:
    print("✓ No infinite density values")

## 9. Prepare final output dataset

The final dataset contains only the essential columns for downstream analysis:
- Year
- Mun_Code
- Mun (municipality name)
- Pop_Density

In [ ]:
# Select final output columns
density_df = df_merged[
    ['Year', 'Mun_Code', 'Mun', 'Pop_Density']
].copy()

print(f"Final dataset: {len(density_df)} records")
density_df.head(10)

## 10. Export derived dataset

Population density is exported as a standalone CSV file.
This modular structure allows future integration with other demographic,
agricultural, and land-use indicators without compromising traceability.

In [ ]:
# Export derived indicator
output_file = DERIVED_DIR / "demography_population_density_1996_2024.csv"

density_df.to_csv(output_file, index=False)

print(f"✓ Population density dataset exported to:")
print(f"  {output_file}")

In [ ]:
# Clean up temporary files
import shutil

if temp_dir.exists():
    shutil.rmtree(temp_dir)
    print("✓ Temporary files cleaned up")